In [1]:
#%load_ext autoreload
#%autoreload 0

In [2]:
import functools
import json
import os

import numpy as np
import pandas as pd
from scrapy.utils.project import get_project_settings
from tqdm.notebook import tqdm

from tse.common.ballot_box_files import (BallotBoxFileType,
                                         get_ballot_box_files_map,
                                         read_ballot_box_logs)
from tse.common.pathinfo import PathInfo
from tse.parsers import SectionAuxParser, SectionsConfigParser

In [3]:
#%autoreload

In [4]:
settings = get_project_settings()
plea = settings["PLEA"]
elections = settings["ELECTIONS"]
states= settings["STATES"]

display(plea)
display(elections)
display(' '.join(states))

'406'

['544', '546', '548']

'br ac al am ap ba ce df es go ma mg ms mt pa pb pe pi pr rj rn ro rr rs sc se sp to zz'

In [5]:
@functools.lru_cache(500000)
def load_json(path):
    with open(PathInfo.get_local_path(settings, path), "r") as f:
        return json.load(f)

In [6]:
def get_sections():
    for state in states:
        if state == "br":
            continue

        section_config_path = PathInfo.get_sections_config_path(plea, state)
        
        config_data = load_json(section_config_path)

        for city, zone, section in SectionsConfigParser.expand_sections(config_data):
            aux_path = PathInfo.get_section_aux_path(plea, state, city, zone, section)
            aux_data = load_json(aux_path)
            hash, hashdate, filenames = SectionAuxParser.get_files(aux_data)
            if hash:
                files_map = get_ballot_box_files_map(filenames)
                yield ((state, city, zone, section), (hash, files_map))

all_section_files = dict(tqdm(get_sections(), total=472024))

  0%|          | 0/472024 [00:00<?, ?it/s]

In [12]:
key = ("ac", "1066", "4", "77")
hash, files = all_section_files[key]
log_path = PathInfo.get_ballot_box_file_path(plea, *key, hash, files[BallotBoxFileType.LOG])
log_path

'arquivo-urna/406/dados/ac/01066/0004/0077/395459446c754b34572b56304a706a6a413454646f6f5a6f5664426f5169564241506566444932644f75493d/o00406-0106600040077.logjez'

In [29]:
logs = dict(read_ballot_box_logs(PathInfo.get_local_path(settings, log_path)))
df = logs["o00406-0106600040077.logjez"]
df

,timestamp,level,id_ballot_box,app,message,hash
0,2022-09-22 14:11:11,INFO,67305985,LOGD,Início das operações do logd,8205594207814213261
1,2022-09-22 14:11:11,INFO,67305985,LOGD,Urna ligada em 22/09/2022 às 14:09:59,4421815302842738656
2,2022-09-22 14:11:11,INFO,67305985,SCUE,Iniciando aplicação - Oficial - 1º turno,396608720597996448
3,2022-09-22 14:11:11,INFO,67305985,SCUE,Versão da aplicação: 8.26.0.0 - Onça-pintada,17957230827251748098
4,2022-09-22 14:11:13,INFO,67305985,SCUE,Urna operando com rede elétrica,15157650888632883953
...,...,...,...,...,...,...
5356,2022-10-02 15:52:57,INFO,67305985,VOTA,Gerando arquivo de resultado [.ver] + [Início],612076369282010836
5357,2022-10-02 15:52:57,INFO,67305985,VOTA,Gerando arquivo de resultado [.ver] + [Término],14799536291532103694
5358,2022-10-02 15:52:57,INFO,67305985,VOTA,Gerando arquivo de resultado [.chvtp] + [Início],17677498444101028370
5359,2022-10-02 15:52:57,INFO,67305985,VOTA,Gerando arquivo de resultado [.chvtp] + [Término],12710055081428421582


In [33]:
df.id_ballot_box.cat.codes

0       0
1       0
2       0
3       0
4       0
       ..
5356    0
5357    0
5358    0
5359    0
5360    0
Length: 5361, dtype: int8